<a href="https://www.kaggle.com/code/hakankeskin/rfm-analysis-for-e-customer-data?scriptVersionId=121216191" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 id="basics" style="font-family:verdana;"> 
    <center> RFM Analysis of E-Commerce Data
    </center>
</h1>


<div style="width:100%;text-align: center;"> <img align=middle src="https://hivemarketingcloud.com/media/zphnp5zi/rfm-analysis-blog-graphic-01.png?center=0.55126050420168071,0.58738261801222658&mode=crop&width=730&height=467&rnd=133039200171670000" alt="Heat beating" style="height:300px;margin-top:3rem;"> </div>

In this study, a dataset about the E-Commerce is analyzed to find out the RFM metrics of the customers for the company. This dataset content is shared like that;

"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."


<div style="font-size:15px; font-family:verdana;">Main topics of the study can be seen below:<br><br>

<ol>
    <li>Aim of the study</li>
    <li>Understand the data</li>
    <li>Preparation of data</li>
    <li>RFM Metrics</li>
    <li>RFM Scores</li>
    <li>Creating & Analysing RFM Segments</li>
</ol>

</div>

## 1. Aim of the Study

The main purpose of the study find out the customer's transaction attidutes with the 1 year database result. End of the analysis, customer's RFM metrics can be shared with the company.

<div style="width:100%;text-align: center;"> <img align=middle src="https://d35fo82fjcw0y8.cloudfront.net/2018/03/01013239/Header-e1551869702205.png" alt="Heat beating" style="height:300px;margin-top:3rem;"> </div>

## 2. Understand the Data

First of all we should import the libraries that will use during the analysis and segmentation processes.

In [1]:
# Pandas and Datetime libraries should be imported too read and convert the dates to readable formats.

import pandas as pd
import datetime as dt
pd.set_option("display.float_format", lambda x: '%.3f' % x)


In [2]:
# Lets import the dataset

df = pd.read_csv(r"/kaggle/input/ecommerce-data/data.csv", encoding= 'unicode_escape')

In [3]:
# To understand the "check_df" functione can be used to decide the what should we do about the data.

def check_df(dataframe, head=5):
    print("########## Info #############")
    print(dataframe.info())
    print("########## Shape #############")
    print(dataframe.shape)
    print("########## Data Types #############")
    print(dataframe.dtypes)
    print("########## Head of Data #############")
    print(dataframe.head(head))
    print("########## Tail of Data #############")
    print(dataframe.tail(head))
    print("########## Null Values of Data #############")
    print(dataframe.isnull().sum())
    print("########## Describe of the Numerical Datas #############")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

########## Info #############
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None
########## Shape #############
(541909, 8)
########## Data Types #############
InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object
########## Head of Data #############
  InvoiceNo StockCode                          

Before the start the analysis, according to dataset summary, dataset has 8 variables. Lets check them;

1. Invoice No: Special number of Invoice for the each Customer ID
2. StockCode: Special number for the each kind of products.
3. Description: Summary of the products that sell by the company.
4. Quantity: Sell amounts of the product for each order.
5. InvoiceDate: Date of the Invoice
6. Unit Price: Unit price for the each quantity of the product.
7. CustomerID: Special ID for the customers.
8. Country: Customers Country

According to null value of data, the data has 135080 missing value for the Customer ID. These missing values should be eleminated the data in the next chapters.

Also, according to description part for the numerical variables, we can see the "-" UnitPrice or Quantity. It is meaning some kind of orders cancelled by the customers. We should also elimanite these rows to find out the meaningful results during the analysis process.

In [4]:
# To make a RFM analysis we should have 3 metrics; Recency, Frequency and Monetary. In this dataset we do not have Monetary columns. 

# To obtain the Monetary, its mean Total Price, we can use this method: 

df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.550,17850.000,United Kingdom,15.300
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.390,17850.000,United Kingdom,20.340
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.750,17850.000,United Kingdom,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.390,17850.000,United Kingdom,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.390,17850.000,United Kingdom,20.340


## 3. Preparation of the Data

In the second stage, we mentioned about the null values in the Customer ID columns. Now, we should eliminate the null values from the data.

In [5]:
# dropna() command will help to drop the null values from the data.
df.dropna(inplace = True)

# Lets check the data
df.describe().T

# As we can see total amount of data decreased the after the dropping to 406829. But we have still negative Quantity and Total Price values.


,count,mean,std,min,25%,50%,75%,max
Quantity,406829.000,12.061,248.693,-80995.000,2.000,5.000,12.000,80995.000
UnitPrice,406829.000,3.460,69.315,0.000,1.250,1.950,3.750,38970.000
CustomerID,406829.000,15287.691,1713.600,12346.000,13953.000,15152.000,16791.000,18287.000
TotalPrice,406829.000,20.402,427.592,-168469.600,4.200,11.100,19.500,168469.600


In [6]:
# Lets filtered the data from negative values. 

df[df["Quantity"] < 0]
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,406829.000,12.061,248.693,-80995.000,2.000,5.000,12.000,80995.000
UnitPrice,406829.000,3.460,69.315,0.000,1.250,1.950,3.750,38970.000
CustomerID,406829.000,15287.691,1713.600,12346.000,13953.000,15152.000,16791.000,18287.000
TotalPrice,406829.000,20.402,427.592,-168469.600,4.200,11.100,19.500,168469.600


In [7]:
# To make sure any return order not in the dataset. Because of that we should drop the InvoiceNo that contains "C".

df = df[~df["InvoiceNo"].str.contains("C", na = False)]
# After the filtering we can start the analysis of the dataset.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397924.000,13.022,180.420,1.000,2.000,6.000,12.000,80995.000
UnitPrice,397924.000,3.116,22.097,0.000,1.250,1.950,3.750,8142.750
CustomerID,397924.000,15294.315,1713.170,12346.000,13969.000,15159.000,16795.000,18287.000
TotalPrice,397924.000,22.395,309.056,0.000,4.680,11.800,19.800,168469.600


## 4. Calculation RFM Metrics

The main focus of this work to find out the RFM segmentation of the customers for the company. Thus, we need to prepare the data for the RFM calculations. Firstly, we need to decide which variables are used to convert the Recency, Frequency and Monetary values.

In [8]:
# Before the start we should convert the InvoiceDate to date with the datetime library.

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [9]:
# Now, to find out the Recency value of the customers we should know the last order day in the data and make the analysis after the last day + 1 day.

last_day = df["InvoiceDate"].max()
today_date = dt.datetime(last_day.year, last_day.month, last_day.day + 1)


<h1 id="basics" style="font-family:verdana;"> 
    <center> RFM Metrics
    </center>
</h1>

<div style="width:100%;text-align: center;"> <img align=middle src="https://d35fo82fjcw0y8.cloudfront.net/2018/03/01013508/Incontent_image.png" alt="Heat beating" style="height:300px;margin-top:3rem;"> </div>

In [10]:
# Here is the first part for RFM metrics. In this part data is grouped by Customer ID.

# Recency: Analysis date - last order day of the customer.
# Frequency: Total invoice amount the customer
# Monetary: Total price that spent by customer for each orders.

rfm = df.groupby("CustomerID").agg({"InvoiceDate": lambda date: (today_date - date.max()).days,
                                     'InvoiceNo': lambda num: num.nunique(),
                                     'TotalPrice': "sum",
                                          "Description": "sum"})

In [11]:
# Lets change the rfm columns to understandable.

rfm.columns = ["recency", "frequency", "monetary", "description"]
rfm.head()


,recency,frequency,monetary,description
CustomerID,,,,
12346.000,325,1,77183.600,MEDIUM CERAMIC TOP STORAGE JAR
12347.000,2,7,4310.000,BLACK CANDELABRA T-LIGHT HOLDERAIRLINE BAG VIN...
12348.000,75,4,1797.240,72 SWEETHEART FAIRY CAKE CASES60 CAKE CASES DO...
12349.000,18,1,1757.550,PARISIENNE CURIO CABINETSWEETHEART WALL TIDY P...
12350.000,310,1,334.400,CHOCOLATE THIS WAY METAL SIGNMETAL SIGN NEIGHB...


In [12]:
# In this part we should ignoring the monetary values that equal to 0.

rfm = rfm[rfm["monetary"] >0]

<div style="width:100%;text-align: center;"> <img align=middle src="https://www.annexcloud.com/blog/wp-content/uploads1/2021/06/RFM-Analysis-Model-1.png" alt="Heat beating" style="height:300px;margin-top:3rem;"> </div>

## 5. Calculating of RFM Scores

According RFM Analysis Model we should to prepare the segmentions for the each metrics below.

In [13]:
# According RFM Analysis Model we should to prepare the segmentions for the each metrics below.

rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels = [5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method = "first"), 5, labels = [1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = [1,2,3,4,5])
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm["frequency_score"].astype(str))

rfm.head()

,recency,frequency,monetary,description,recency_score,frequency_score,monetary_score,RFM_SCORE
CustomerID,,,,,,,,
12346.000,325,1,77183.600,MEDIUM CERAMIC TOP STORAGE JAR,1,1,5,11
12347.000,2,7,4310.000,BLACK CANDELABRA T-LIGHT HOLDERAIRLINE BAG VIN...,5,5,5,55
12348.000,75,4,1797.240,72 SWEETHEART FAIRY CAKE CASES60 CAKE CASES DO...,2,4,4,24
12349.000,18,1,1757.550,PARISIENNE CURIO CABINETSWEETHEART WALL TIDY P...,4,1,4,41
12350.000,310,1,334.400,CHOCOLATE THIS WAY METAL SIGNMETAL SIGN NEIGHB...,1,1,2,11


In [14]:
# Then we can make understandable each segment definition below.

seg_map = {r'[1-2][1-2]': "hibernating",
        r'[1-2][3-4]': "at_Risk",
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential,loyalists',
        r'5[4-5]': 'champions'
          }

# Assign of the segments
rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex = True)
rfm.head()

,recency,frequency,monetary,description,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,,
12346.000,325,1,77183.600,MEDIUM CERAMIC TOP STORAGE JAR,1,1,5,11,hibernating
12347.000,2,7,4310.000,BLACK CANDELABRA T-LIGHT HOLDERAIRLINE BAG VIN...,5,5,5,55,champions
12348.000,75,4,1797.240,72 SWEETHEART FAIRY CAKE CASES60 CAKE CASES DO...,2,4,4,24,at_Risk
12349.000,18,1,1757.550,PARISIENNE CURIO CABINETSWEETHEART WALL TIDY P...,4,1,4,41,promising
12350.000,310,1,334.400,CHOCOLATE THIS WAY METAL SIGNMETAL SIGN NEIGHB...,1,1,2,11,hibernating


In [15]:
# To understand the distribution of the each segmentation we can use the groupby.

print(rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"]))



                    recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       52.312   352     1.162   352  471.994   352
at_Risk             152.786   593     2.879   593 1084.535   593
cant_loose          131.968    63     8.381    63 2796.156    63
champions             5.362   633    12.417   633 6857.935   633
hibernating         216.605  1071     1.102  1071  488.643  1071
loyal_customers      32.608   819     6.480   819 2864.248   819
need_attention       51.428   187     2.326   187  897.628   187
new_customers         6.429    42     1.000    42  388.213    42
potential,loyalists  16.399   484     2.010   484 1041.222   484
promising            22.511    94     1.000    94  294.008    94


In [16]:
# To make the readable of the data we can assing the index and obtain the final dataframe.

rfm[rfm["segment"] == "new_customers"].index

rfm.head()


,recency,frequency,monetary,description,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,,
12346.000,325,1,77183.600,MEDIUM CERAMIC TOP STORAGE JAR,1,1,5,11,hibernating
12347.000,2,7,4310.000,BLACK CANDELABRA T-LIGHT HOLDERAIRLINE BAG VIN...,5,5,5,55,champions
12348.000,75,4,1797.240,72 SWEETHEART FAIRY CAKE CASES60 CAKE CASES DO...,2,4,4,24,at_Risk
12349.000,18,1,1757.550,PARISIENNE CURIO CABINETSWEETHEART WALL TIDY P...,4,1,4,41,promising
12350.000,310,1,334.400,CHOCOLATE THIS WAY METAL SIGNMETAL SIGN NEIGHB...,1,1,2,11,hibernating


## 6. Creating & Analysing RFM Segments


In [17]:
# In this part we can see the total segment number.

for segment in rfm["segment"].unique():
    shape = rfm[(rfm["segment"] == segment)].shape
    print(f"Total {segment} value is {shape[0]}")



Total hibernating value is 1071
Total champions value is 633
Total at_Risk value is 593
Total promising value is 94
Total loyal_customers value is 819
Total potential,loyalists value is 484
Total about_to_sleep value is 352
Total need_attention value is 187
Total new_customers value is 42
Total cant_loose value is 63


## 7. Conclusion

End of the analysis we obtain the useful results for the company according to customer datas. So what kind of things done in this work, lets see:

1. Firstly we tried to understand the data and check.
2. After the checking data, some missing values and non useful data observed.
3. Before to start the RFM analysis, these values or datas dropped from dataframe and prepared for the analysis.
4. Aim of the analysis is to find out the behavior of the customers according to their orders.
5. In this part, according to RFM analysis basis, data is filtered and transformed to for the Recency, Frequency and Monetary.
6. These 3 variables are the fundemental of the RFM analysis kind a keys.
7. After the obtain RFM metrics, each customer behavior defined with RFM results like a numerical segmentation.
8. Numerical segmentation is given meaningful observation but to make it much more clear, these segmentations changed as category according to
RFM Grid.
9. End of the analysis we filtered the customers according to their classification or segmentation and after this part, company can know who is the most important or critical customers in the dataset.

Also, any user can filtered data according to these results and obtain meaningful summary.